In [1]:
import ee
import geemap

from src.utils.datautils import monitor_task

In [2]:
# ! earthengine create folder projects/nb-lidar/assets/exp3

# Experiment 3:
3 Class Problem: Non Wetland, Water, Wetland
Remove all Auxiliary Datasets i.e. Fourier Transform and Alos Palsar form the stack

stack will contain DataCube(Sentinel-2) and Sentinel - 1 compoistes for now + elevation prodcuts - elevation band

Training data will has been pre generated and saved in the form gee table assets, features will be extracted from the stack and used to train the model

In [3]:
ee.Initialize(project="nb-lidar")

In [4]:
def remove_band(band_names: ee.List | list[str], name: str | list[str]):
    if isinstance(band_names, list):
        band_names = ee.List(band_names)
    
    if isinstance(name, str):
        return band_names.remove(name)
    return band_names.removeAll(name)

In [5]:
# DEFINE CONSTANTS
folder = "exp3"
aoi_id = "projects/nb-lidar/assets/aoi_nb_south"
input_data_id = "projects/nb-lidar/assets/nb_south_3_class_bal"
terrain_type = "dtm"

In [6]:
aoi = ee.FeatureCollection(aoi_id).geometry()
input_data = ee.FeatureCollection(input_data_id)

In [7]:
from src import image_processing as proc

s1 = proc.fetch_and_proecss_s1_seasonal(aoi)
s2 = proc.fetch_and_proeces_s2_sr_seasonal(aoi, ndvi=True)
al = proc.fetch_and_process_alos()
ft = proc.fetch_fourier_transform()
ta = proc.fetch_terrain(terrain_type)

elevation_bands = remove_band(ta.bandNames(), 'Elevation')

stack = ee.Image.cat(s1, s2, al, ft, ta.select(elevation_bands))

In [8]:
# Extract Features
features = stack.sampleRegions(
    collection=input_data,
    tileScale=16,
    scale=10
)

In [9]:
# split features into train and test
train = features.filter('random < 0.7')
test = features.filter('random >= 0.7')

In [10]:
# Train the Random Forest Model
rf = ee.classifier.Classifier.smileRandomForest(1000).train(train, 'class_name', stack.bandNames())

In [11]:
# assess the model
# assessment
validated = test.classify(rf)
error_matrix = validated.errorMatrix("class_name", "classification", order=[1, 2, 3])

In [12]:
error_matrix

In [13]:
error_matrix.accuracy().multiply(100).format("%.2f")

In [14]:
print("Conusmers Accuracy")
print("Non Wetland, Water, Wetland")
error_matrix.consumersAccuracy().toList().flatten().map(lambda x: ee.Number(x).multiply(100).format("%.2f"))

Conusmers Accuracy
Non Wetland, Water, Wetland


In [15]:
print("Producers Accuracy")
print("Non Wetland, Water, Wetland")
error_matrix.producersAccuracy().toList().flatten().map(lambda x: ee.Number(x).multiply(100).format("%.2f"))

Producers Accuracy
Non Wetland, Water, Wetland


In [16]:
# make prediction using the model
prediction = stack.classify(rf)

In [17]:
Map = geemap.Map()

Map.addLayer(prediction, {'palette': ['grey', 'blue', 'green'], 'min': 1, 'max': 3}, 'Prediction')
Map.addLayerControl()
Map.centerObject(aoi, zoom=8)

Map

Map(center=[45.74809742262684, -65.99999969409251], controls=(WidgetControl(options=['position', 'transparent_…